# 🔗 Phishing URL Detection — Feature Analysis
### Phase 4 | Logistic Regression + Random Forest
---
📂 **Dataset:** `phishing_url_ml_ready.csv`
- 48,812 URLs — perfectly balanced (50% phishing / 50% legitimate)
- 27 pre-extracted features (URL length, HTTPS, @ symbol, domain entropy, etc.)
- Label: `1` = Phishing, `0` = Legitimate

> ✅ No GPU needed for this phase — runs on CPU just fine!
---

## ✅ Step 1 — Install & Import Libraries

In [ ]:
!pip install scikit-learn seaborn -q
print('✅ Libraries ready!')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix, roc_auc_score, roc_curve
)

print('✅ All imports done!')

## ✅ Step 2 — Upload Dataset

In [ ]:
from google.colab import files
print('📂 Upload phishing_url_ml_ready.csv')
uploaded = files.upload()
print('✅ Uploaded:', list(uploaded.keys()))

## ✅ Step 3 — Load & Explore Data

In [ ]:
df = pd.read_csv('phishing_url_ml_ready.csv')

print('Shape:', df.shape)
print('\nLabel distribution:')
print(df['label'].value_counts())
print('\nFeature overview:')
display(df.describe().round(3))

In [ ]:
# Compare key features between phishing and legitimate URLs
key_features = ['url_length', 'is_https', 'has_ip_address',
                'num_at_signs', 'num_hyphens', 'domain_entropy',
                'has_suspicious_tld', 'num_suspicious_words']

comparison = df.groupby('label')[key_features].mean().round(3)
comparison.index = ['Legitimate (0)', 'Phishing (1)']

print('📊 Average feature values by class:')
display(comparison.T)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 9))
axes = axes.flatten()

for i, feature in enumerate(key_features):
    for label, color, name in [(0, 'steelblue', 'Legitimate'), (1, 'tomato', 'Phishing')]:
        axes[i].hist(
            df[df['label'] == label][feature],
            bins=30, alpha=0.6, color=color, label=name
        )
    axes[i].set_title(feature, fontsize=11)
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Count')
    axes[i].legend(fontsize=8)

plt.suptitle('Feature Distributions: Phishing vs Legitimate URLs', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

## ✅ Step 4 — Prepare Data for ML

In [ ]:
# Separate features and label
X = df.drop(columns=['label'])
y = df['label']

feature_names = X.columns.tolist()
print(f'Features: {len(feature_names)}')
print(f'Samples:  {len(X)}')

# Train / Test split — 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f'\nTrain: {len(X_train)} | Test: {len(X_test)}')

# Scale features — required for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print('✅ Data ready!')

## ✅ Step 5 — Model 1: Logistic Regression

In [ ]:
print('Training Logistic Regression...')
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train)

lr_preds = lr_model.predict(X_test_scaled)
lr_probs = lr_model.predict_proba(X_test_scaled)[:, 1]

lr_acc = accuracy_score(y_test, lr_preds)
lr_auc = roc_auc_score(y_test, lr_probs)

print(f'\n✅ Logistic Regression Results:')
print(f'   Accuracy:  {lr_acc:.4f} ({lr_acc*100:.2f}%)')
print(f'   ROC-AUC:   {lr_auc:.4f}')
print(f'\n{classification_report(y_test, lr_preds, target_names=["Legitimate", "Phishing"])}')

## ✅ Step 6 — Model 2: Random Forest

In [ ]:
print('Training Random Forest... (takes ~1 min)')
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    random_state=42,
    n_jobs=-1  # use all CPU cores
)
rf_model.fit(X_train, y_train)  # Random Forest doesn't need scaling

rf_preds = rf_model.predict(X_test)
rf_probs = rf_model.predict_proba(X_test)[:, 1]

rf_acc = accuracy_score(y_test, rf_preds)
rf_auc = roc_auc_score(y_test, rf_probs)

print(f'\n✅ Random Forest Results:')
print(f'   Accuracy:  {rf_acc:.4f} ({rf_acc*100:.2f}%)')
print(f'   ROC-AUC:   {rf_auc:.4f}')
print(f'\n{classification_report(y_test, rf_preds, target_names=["Legitimate", "Phishing"])}')

## ✅ Step 7 — Compare Both Models

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# ── Confusion Matrix: Logistic Regression ───────────────────────
cm_lr = confusion_matrix(y_test, lr_preds)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Legitimate', 'Phishing'],
            yticklabels=['Legitimate', 'Phishing'],
            annot_kws={'size': 13})
axes[0].set_title(f'Logistic Regression\nAccuracy: {lr_acc*100:.2f}%', fontsize=12)
axes[0].set_ylabel('Actual')
axes[0].set_xlabel('Predicted')

# ── Confusion Matrix: Random Forest ─────────────────────────────
cm_rf = confusion_matrix(y_test, rf_preds)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['Legitimate', 'Phishing'],
            yticklabels=['Legitimate', 'Phishing'],
            annot_kws={'size': 13})
axes[1].set_title(f'Random Forest\nAccuracy: {rf_acc*100:.2f}%', fontsize=12)
axes[1].set_ylabel('Actual')
axes[1].set_xlabel('Predicted')

# ── ROC Curves ──────────────────────────────────────────────────
fpr_lr, tpr_lr, _ = roc_curve(y_test, lr_probs)
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs)

axes[2].plot(fpr_lr, tpr_lr, color='steelblue', lw=2,
             label=f'Logistic Regression (AUC = {lr_auc:.3f})')
axes[2].plot(fpr_rf, tpr_rf, color='tomato', lw=2,
             label=f'Random Forest (AUC = {rf_auc:.3f})')
axes[2].plot([0, 1], [0, 1], 'k--', lw=1)
axes[2].set_title('ROC Curve Comparison', fontsize=12)
axes[2].set_xlabel('False Positive Rate')
axes[2].set_ylabel('True Positive Rate')
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ── Summary table ───────────────────────────────────────────────
print('\n' + '='*45)
print('         MODEL COMPARISON SUMMARY')
print('='*45)
print(f'{"Model":<25} {"Accuracy":>10} {"AUC":>10}')
print('-'*45)
print(f'{"Logistic Regression":<25} {lr_acc*100:>9.2f}% {lr_auc:>10.4f}')
print(f'{"Random Forest":<25} {rf_acc*100:>9.2f}% {rf_auc:>10.4f}')
print('='*45)
winner = 'Random Forest' if rf_acc > lr_acc else 'Logistic Regression'
print(f'🏆 Best model: {winner}')

## ✅ Step 8 — Feature Importance (Random Forest)

In [ ]:
importances = pd.Series(rf_model.feature_importances_, index=feature_names)
importances = importances.sort_values(ascending=True)

plt.figure(figsize=(10, 8))
colors = ['tomato' if v > importances.median() else 'steelblue'
          for v in importances.values]
importances.plot(kind='barh', color=colors, edgecolor='black', linewidth=0.5)
plt.title('Feature Importance — Random Forest', fontsize=14)
plt.xlabel('Importance Score')
plt.tight_layout()
plt.show()

print('\nTop 5 most important features:')
for i, (feat, score) in enumerate(importances.sort_values(ascending=False).head(5).items(), 1):
    print(f'  {i}. {feat}: {score:.4f}')

## ✅ Step 9 — Live URL Prediction

In [ ]:
import re
import math
from urllib.parse import urlparse

def extract_features(url):
    """Extract the same 27 features used during training."""
    f = {}
    f['url_length']           = len(url)
    f['num_dots']             = url.count('.')
    f['num_hyphens']          = url.count('-')
    f['num_underscores']      = url.count('_')
    f['num_slashes']          = url.count('/')
    f['num_question_marks']   = url.count('?')
    f['num_equal_signs']      = url.count('=')
    f['num_at_signs']         = url.count('@')
    f['num_ampersands']       = url.count('&')
    f['num_percent']          = url.count('%')
    f['num_digits']           = sum(c.isdigit() for c in url)
    f['digit_ratio']          = f['num_digits'] / len(url) if len(url) > 0 else 0
    suspicious_words = ['login','signin','verify','secure','account',
                        'update','banking','confirm','password','pay',
                        'free','lucky','win','bonus','click']
    f['num_suspicious_words'] = sum(w in url.lower() for w in suspicious_words)
    f['has_ip_address']       = int(bool(re.search(r'(https?://)?(\d{1,3}\.){3}\d{1,3}', url)))

    try:
        parsed = urlparse(url if url.startswith('http') else 'http://' + url)
    except:
        parsed = urlparse('')

    f['is_https']             = int(parsed.scheme == 'https')
    netloc                    = parsed.netloc or ''
    domain                    = netloc.split(':')[0]
    f['domain_length']        = len(domain)
    f['num_subdomains']       = max(0, len(domain.split('.')) - 2)
    parts                     = domain.split('.')
    tld                       = parts[-1].lower() if len(parts) >= 2 else ''
    suspicious_tlds           = ['ru','xyz','tk','ml','ga','cf','gq','top',
                                  'work','click','link','info','biz','online']
    f['has_suspicious_tld']   = int(tld in suspicious_tlds)
    path                      = parsed.path or ''
    f['path_length']          = len(path)
    f['path_depth']           = path.count('/')
    f['has_exe_extension']    = int(bool(re.search(r'\.(exe|sh|bat|cmd|msi|ps1|vbs|js)$', path.lower())))
    f['has_php']              = int('.php' in path.lower())
    query                     = parsed.query or ''
    f['query_length']         = len(query)
    f['num_query_params']     = query.count('&') + 1 if query else 0
    port                      = parsed.port
    f['has_non_standard_port']= int(port is not None and port not in [80, 443])
    if domain:
        probs = [domain.count(c) / len(domain) for c in set(domain)]
        f['domain_entropy']   = -sum(p * math.log2(p) for p in probs if p > 0)
    else:
        f['domain_entropy']   = 0
    f['tld_encoded']          = hash(tld) % 100  # simple encoding
    return f


def predict_url(url):
    """Predict whether a URL is phishing or legitimate."""
    features = extract_features(url)
    X_new = pd.DataFrame([features])[feature_names]
    X_new_scaled = scaler.transform(X_new)

    # Random Forest prediction (best model)
    pred  = rf_model.predict(X_new)[0]
    proba = rf_model.predict_proba(X_new)[0]

    label = '🚨 PHISHING' if pred == 1 else '✅ LEGITIMATE'
    print(f'URL:        {url}')
    print(f'Prediction: {label}')
    print(f'Confidence: {proba.max()*100:.2f}%')
    print(f'Legitimate: {proba[0]*100:.2f}%  |  Phishing: {proba[1]*100:.2f}%')
    print('-' * 65)


print('='*65)
print('          LIVE URL PHISHING DETECTION DEMO')
print('='*65)

predict_url('https://www.google.com')
predict_url('http://paypal-secure-login.verify-account.tk/update')
predict_url('https://github.com/features')
predict_url('http://192.168.1.1:8080/admin/login.php')
predict_url('http://free-iphone-winner.click/claim?user=you&prize=yes')

In [ ]:
# ── Try YOUR OWN URL here ────────────────────────────────────────
my_url = "https://paste-any-url-here.com"

predict_url(my_url)

## ✅ Step 10 — Save Both Models

In [ ]:
import pickle

# Save Random Forest
with open('url_rf_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

# Save Logistic Regression
with open('url_lr_model.pkl', 'wb') as f:
    pickle.dump(lr_model, f)

# Save scaler (needed for Logistic Regression predictions)
with open('url_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print('✅ Models saved!')

# Zip and download all
!zip url_models.zip url_rf_model.pkl url_lr_model.pkl url_scaler.pkl

from google.colab import files
files.download('url_models.zip')
print('✅ Download started!')

---
## 🎉 Phase 4 Complete!

| Step | Task | Status |
|------|------|--------|
| 1 | Install libraries | ✅ |
| 2 | Upload dataset | ✅ |
| 3 | Load & explore (48,812 URLs) | ✅ |
| 4 | Prepare features + split 80/20 | ✅ |
| 5 | Train Logistic Regression | ✅ |
| 6 | Train Random Forest | ✅ |
| 7 | Compare models + ROC curve | ✅ |
| 8 | Feature importance chart | ✅ |
| 9 | Live URL prediction function | ✅ |
| 10 | Save & download models | ✅ |

**Expected results:**
- Logistic Regression: ~88–92% accuracy
- Random Forest: ~96–98% accuracy

**Files saved:**
- `url_rf_model.pkl` — Random Forest model
- `url_lr_model.pkl` — Logistic Regression model  
- `url_scaler.pkl` — Feature scaler
---